In this BONUS point, we start again from the RAW data, because we will need some features that we discarder when saving the processed data.

### Preprocessing

In [1]:
import sys
!{sys.executable} -m pip install sklearn pandas

Looking in indexes: https://pypi.python.org/simple/


In [2]:
import time
import pandas as pd
import datetime
from sklearn.preprocessing import StandardScaler
from IPython.display import display, HTML


In [3]:
CSV_PATH = r"..\data\data_for_student_case.csv"
data_df = pd.read_csv(CSV_PATH)
print(data_df.shape)

(290382, 17)


In [4]:
# same preprocessing like before
data_df = data_df[data_df['simple_journal'] != 'Refused']
data_df['cvcresponsecode'] = data_df['cvcresponsecode'].apply(lambda cvc: 3 if cvc >= 3 else cvc)
data_df = data_df.loc[~data_df['cardverificationcodesupplied'].isna()]
data_df['label'] = data_df['simple_journal'].apply(lambda label: 1 if label == 'Chargeback' else 0)
data_df = data_df.drop(columns = ['simple_journal'])

# we keep creation_date this time.

conversion_rate = {"AUD": 0.626093,
                    "NZD": 0.591501,
                    "SEK": 0.0935468,
                    "GBP": 1.16536,
                    "MXN": 0.0467946}

data_df['amount'] = data_df[['amount', 'currencycode']].apply(lambda row: row['amount']*conversion_rate[row['currencycode']], axis=1)

for col_name in ['issuercountrycode', 'txvariantcode', 'currencycode', 'shoppercountrycode', 'shopperinteraction', 'accountcode', 'mail_id', 'ip_id', 'card_id']:
    unique_values = data_df[col_name].unique()
    mapped_values = dict(zip(unique_values, range(len(unique_values))))
    data_df[col_name] = data_df[col_name].apply(lambda old_value: mapped_values[old_value])

data_df = data_df.drop(columns = ['currencycode'])



In [5]:
print(data_df.shape)
data_df = data_df.drop(columns=['txid', 'bin'])

(223842, 16)


In [6]:
data_df['creationdate'] = pd.to_datetime(data_df['creationdate'],format='%Y-%m-%d %H:%M:%S')
data_df['creationmonth'] = data_df.creationdate.dt.month
data_df['creationweekday'] = data_df.creationdate.dt.weekday
data_df['creationday'] = data_df.creationdate.dt.day

data_df['bookingdate'] = pd.to_datetime(data_df['bookingdate'],format='%Y-%m-%d %H:%M:%S')
print(list(data_df.columns))

['bookingdate', 'issuercountrycode', 'txvariantcode', 'amount', 'shoppercountrycode', 'shopperinteraction', 'cardverificationcodesupplied', 'cvcresponsecode', 'creationdate', 'accountcode', 'mail_id', 'ip_id', 'card_id', 'label', 'creationmonth', 'creationweekday', 'creationday']


In [7]:
data_df.head()

,bookingdate,issuercountrycode,txvariantcode,amount,shoppercountrycode,shopperinteraction,cardverificationcodesupplied,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id,label,creationmonth,creationweekday,creationday
0,2015-11-09 14:26:51,0,0,3032.29008,0,0,True,0,2015-07-01 23:03:11,0,0,0,0,1,7,2,1
1,2015-11-09 14:27:38,0,0,2101.07754,0,0,True,0,2015-07-02 04:50:55,0,1,1,1,1,7,3,2
2,2015-11-23 16:34:16,0,0,7014.51054,0,0,True,0,2015-07-02 14:30:28,0,2,2,2,1,7,3,2
3,2015-11-23 16:34:51,0,0,5142.72654,0,0,True,0,2015-07-03 07:53:37,0,3,3,3,1,7,4,3
4,2015-11-09 14:26:08,0,1,4206.83454,0,0,True,0,2015-07-08 18:35:35,0,4,4,4,1,7,2,8


### Aggregations

In [9]:
data_df = data_df.sort_values('creationdate')
final_df = data_df.copy()

First idea: We want to check whether of not using previously detected frauds (grouping by card/ip/mail) can help the detection of fraudolent cases in the future.   
However, we are only allowed to use this information for new cases whose 'creationdate' happens after the 'bookingdate' of the fraudolent case.   
In English language this translates into 'We can use our knowledge of a fraudolent case only for detecting cases which happen after we acquire such knowledge'.

In [10]:
fraudolent_df = data_df[data_df['label'] == 1]
fraudolent_df.shape
final_df.head()

,bookingdate,issuercountrycode,txvariantcode,amount,shoppercountrycode,shopperinteraction,cardverificationcodesupplied,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id,label,creationmonth,creationweekday,creationday
58673,2015-07-01 22:57:05,22,5,5937.5092,19,0,True,1,2015-07-01 00:00:41,2,32580,32583,32765,0,7,2,1
58674,2015-07-01 22:49:20,22,5,5704.4372,19,0,True,1,2015-07-01 00:00:42,2,32581,32584,32766,0,7,2,1
58675,2015-07-01 22:57:06,22,5,5471.3652,19,0,True,1,2015-07-01 00:00:48,2,32582,32585,32767,0,7,2,1
58676,2015-07-01 22:57:05,22,5,6520.1892,19,0,True,1,2015-07-01 00:00:58,2,32583,32586,32768,0,7,2,1
58677,2015-07-01 22:57:06,22,5,8151.6932,19,0,True,1,2015-07-01 00:01:11,2,32584,32587,32769,0,7,2,1


In [12]:
for _, row in fraudolent_df.iterrows():
    current_booking = row['bookingdate']
    card_id = row['card_id']
    ip_id = row['ip_id']
    mail_id = row['mail_id']
    
    # affected rows for which we can use our knowledge about previous frauds
    affected_rows_by_card = final_df[(final_df['creationdate'] > current_booking) & (final_df['card_id'] == card_id)]
    if affected_rows_by_card.shape[0]:
        print("By card")
        print(f"Reported at: {current_booking}")
        display(affected_rows_by_card)
        
    affected_rows_by_ip = final_df[(final_df['creationdate'] > current_booking) & (final_df['ip_id'] == ip_id)]
    if affected_rows_by_ip.shape[0]:
        print("By ip")
        print(f"Reported at: {current_booking}")
        display(affected_rows_by_ip)
    
    affected_rows_by_mail = final_df[(final_df['creationdate'] > current_booking) & (final_df['mail_id'] == mail_id)]
    if affected_rows_by_mail.shape[0]:
        print("By mail")
        print(f"Reported at: {current_booking}")
        display(affected_rows_by_mail)

By mail
Reported at: 2015-08-14 05:45:48


,bookingdate,issuercountrycode,txvariantcode,amount,shoppercountrycode,shopperinteraction,cardverificationcodesupplied,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id,label,creationmonth,creationweekday,creationday
153894,2015-09-03 02:15:19,22,5,28551.320,19,0,True,1,2015-09-01 16:20:26,2,32558,41779,114149,0,9,1,1
194697,2015-10-01 17:53:09,22,5,5244.120,19,1,False,0,2015-09-30 19:06:28,2,32558,41779,114149,0,9,2,30
235525,2015-10-30 00:02:19,22,5,14567.000,19,1,False,0,2015-10-29 00:24:24,2,32558,41283,114149,0,10,3,29
238336,2015-11-02 18:58:12,22,5,9672.488,19,1,False,0,2015-10-30 22:33:59,2,32558,178495,114149,0,10,4,30


By ip
Reported at: 2015-10-09 05:40:10


,bookingdate,issuercountrycode,txvariantcode,amount,shoppercountrycode,shopperinteraction,cardverificationcodesupplied,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id,label,creationmonth,creationweekday,creationday
289549,2015-10-26 18:50:04,21,5,6819.56172,26,0,True,1,2015-10-23 19:53:50,3,197312,179623,199246,0,10,4,23


By ip
Reported at: 2015-10-09 05:40:10


,bookingdate,issuercountrycode,txvariantcode,amount,shoppercountrycode,shopperinteraction,cardverificationcodesupplied,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id,label,creationmonth,creationweekday,creationday
289549,2015-10-26 18:50:04,21,5,6819.56172,26,0,True,1,2015-10-23 19:53:50,3,197312,179623,199246,0,10,4,23


All these affected rows have label 0 (no-fraud). Therefore, using this type of aggregation does not provide any meaningful insight for the classification (For the dataset in use).

A second idea is to aggregate per card/id/ip the number of transactions that have been made in the same day before the current transaction